# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

In [ ]:
import pandas as pd

In [ ]:
dt = pd.read_json('addres-book.json')
dt['email']

2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

In [ ]:
dt['phones']

3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

In [ ]:
from bs4 import BeautifulSoup as bs 
with open('addres-book-q.xml', 'r') as f:
    book = bs(f, 'xml')
    
book

## Лабораторная работа №4

### JSON

In [ ]:
import json

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [ ]:
with open('contributors_sample.json', 'r') as f:
    data = json.load(f)
data[:3]

1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [ ]:
res = set([l.split('@')[1] for l in [i['mail'] for i in data]])
print(res)

In [ ]:
res = set([l.partition('@')[2] for l in [i['mail'] for i in data]])
print(res)

1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [ ]:
def search(name):
    for i in data: 
        if i['username'] == name:
            return i
        else: 
            raise ValueError
print(search('uhebert'))
print(search('Stiles'))

1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [ ]:
sex = [i['sex'] for i in data]
print(f'female: {sex.count("F")}, male: {sex.count("M")}')

1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json('contributors_sample.json')
contributors = df[['id', 'username', 'sex']]
contributors

1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [ ]:
recipes = pd.read_csv('recipes_sample.csv')
res = pd.merge(contributors, recipes, left_on = 'id', right_on = 'contributor_id', how = 'right')
res.head()

In [ ]:
res['id_x'].isna().value_counts()[1]

### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [ ]:
jobs = set(sum([i['jobs'] for i in data], []))
ppl = {j: [] for j in jobs}
for i in data: 
    for j in jobs: 
        if j in i['jobs']: 
            ppl[j].append(i['username'])
ppl

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [ ]:
import pickle 

In [ ]:
with open('job_people.pickle', 'wb') as f:
    pickle.dump(ppl, f, protocol=pickle.HIGHEST_PROTOCOL)
with open('job_people.json', 'w', encoding = 'utf-8') as f:
    json.dump(ppl, f, indent = 2)

In [ ]:
from os import stat
stat('job_people.json').st_size, stat('job_people.pickle').st_size

2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [ ]:
df = pd.read_pickle('job_people.pickle')
df == ppl

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [ ]:
from bs4 import BeautifulSoup as bs 
with open('steps_sample.xml', 'r') as f:
    sample = bs(f, 'xml')
    
sample

In [ ]:
steps_sample = {}
idd = sample.find('id')
while idd: 
    steps_sample[idd.text] = [i.text for i in idd.find_next('steps').find_all('step')]
    idd = idd.find_next('id')

In [ ]:
steps_sample

In [ ]:
with open('steps_sample.json', 'w', encoding = 'utf-8') as f:
    json.dump(steps_sample, f, indent = 2)

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [ ]:
recipes_sample = {}
idd = sample.find('id')
while idd: 
    s_len = len(idd.find_next('steps').find_all('step'))
    if s_len not in recipes_sample:
        recipes_sample[s_len] = [idd]
    else:
        recipes_sample[s_len].append(idd)
        idd = idd.find_next('id')
recipes_sample

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [ ]:
def get_recipes():
    idd = sample.find('id')
    while idd: 
        findsteps = idd.find_next('steps')
        if findsteps('step', {'has_minutes': 1}) or findsteps.find('step', {'has_hours': 1}):
            yield idd.text
        idd = idd.find_next('id')
[i for i in get_recipes()]

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [ ]:
recipes = pd.read_csv('recipes_sample.csv')
recipes

In [ ]:
recipes_len = {int(i.find('id').get_text()): len(i.find_all('step')) for i in sample.find_all('recipes')}
for index, row in recipes.iterrows():
    if pd.isnull(recipes['n_steps'][index]):
        recipes.loc[index, 'n_steps'] = recipes_len[recipes['id'][index]]
recipes

In [ ]:
recipes_len = {}
idd = sample.find('id')
while idd: 
    s_len = len(idd.find_next('steps').find_all('step'))
    recipes_len[idd] = s_len
    idd = idd.find_next('idd')

In [ ]:
recipes['n_steps'] = recipes['n_steps'].fillna(recipes['id'].map(recipes_len))

In [ ]:
recipes.head()

3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [ ]:
recipes['n_steps'].isna().sum()

In [ ]:
print(f'Кол-во пустых значений:{recipes[recipes["n_steps"].isna()].shape[0]}')
recipes['n_steps'] = recipes['n_steps'].astype(int)
recipes